In [13]:
import sys

sys.path.append(r'C:\repos\ra2ce_multi_network') # path to the package
sys.path.append(r'C:\repos\ra2ce') # path to the package

import geopandas as gpd
import pickle

import pyproj
from networkx import MultiDiGraph
from pathlib import Path
from snkit import Network

from ra2ce.graph.origins_destinations import add_od_nodes
from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.graph.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.graph.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.graph.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper
from ra2ce.graph.network_config_data.enums.network_type_enum import NetworkTypeEnum

from ra2ce_multi_network.simplify_rail import _nx_to_network
import ra2ce.graph.networks_utils as nut

In [18]:
# Defining ini variables
root_folder = Path(
    r'path_to_od_and_polygon_files')

polygon_path = root_folder.joinpath(f'polygon.geojson')
od_file = root_folder.joinpath('od_nodes.geojson')
od_gdf = gpd.read_file(od_file)

In [15]:
# Setting up the config data
config_data = NetworkConfigData(
    static_path=root_folder.joinpath('static'),
    crs=pyproj.CRS("EPSG:4326")
)

road_types = [
        'motorway', 'motorway_link', 'primary', 'primary_link', 'secondary', 'secondary_link', 'trunk', 'trunk_link'
    ]

config_data.network = NetworkSection(
    network_type=NetworkTypeEnum.get_enum('drive'),
    road_types=[RoadTypeEnum.get_enum(road_type) for road_type in road_types],
    polygon=polygon_path
)

In [ ]:
# Download the graph from osm and save

osm_network_wrapper = OsmNetworkWrapper(config_data=config_data)
complex_graph = osm_network_wrapper.get_clean_graph_from_osm()
complex_graph.graph["crs"] = pyproj.CRS("EPSG:4326")
with open(root_folder.joinpath(f'road_complex_graph.p'), 'wb') as f:
    pickle.dump(complex_graph, f)

In [ ]:
# Save the outputs
od_gdf.to_file(od_file, driver="GeoJSON")

complex_graph_exporter = MultiGraphNetworkExporter(
    basename=f'road_complex_graph_od_mapped',
    export_types=['pickle', 'gpkg'])
complex_graph_exporter.export_to_gpkg(output_dir=root_folder.joinpath(f'output_graph'),
                                      export_data=complex_graph)
complex_graph_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'output_graph'),
                                        export_data=complex_graph)

In [6]:
# Load graphs and networks prepared befor
# Approach 1

od_gdf = gpd.read_file(od_file)

with open(root_folder.joinpath(f'road_complex_graph_od_mapped.p'), 'rb') as f:
    complex_graph = pickle.load(f)

complex_road_network = Network(
        nodes=_nx_to_network(complex_graph).nodes,
        edges=_nx_to_network(complex_graph).edges
    )

# Approach 2

# complex_road_network = Network(
#     nodes=gpd.read_file(root_folder.joinpath(f'road_complex_graph_od_mapped_nodes.gpkg')),
#     edges=gpd.read_file(root_folder.joinpath(f'road_complex_graph_od_mapped_edges.gpkg'))
# )

In [6]:
# Map origin-destinations the complex graph

if not isinstance(complex_graph, MultiDiGraph):
    complex_graph = MultiDiGraph(complex_graph)

od_gdf, complex_graph = add_od_nodes(od=gpd.read_file(od_file), graph=complex_graph, crs=pyproj.CRS("EPSG:4326"))

Adding Origin-Destination nodes to graph: 5it [00:00, 34.34it/s]


In [7]:
# Visualise

road_map = complex_road_network.edges.explore(tiles="CartoDB positron")
# road_map = od_gdf.explore(m=road_map, color='springgreen')
road_nodes_od_mapped = complex_road_network.nodes[~complex_road_network.nodes['od_id'].isna()]

In [8]:
road_nodes_od_mapped.explore(m=road_map, color='darkgreen')

: 